<a href="https://colab.research.google.com/github/jhwnoh/UST2024Fall_MaterialsInformatics/blob/main/Week10_Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
TorchModule = {'linear':nn.Linear,
               'conv1d':nn.Conv1d,
               'conv2d':nn.Conv2d,
               'conv3d':nn.Conv3d,
               'convtrans1d':nn.ConvTranspose1d,
               'convtrans2d':nn.ConvTranspose2d,
               'convtrans3d':nn.ConvTranspose3d,
               'maxpool1d':nn.MaxPool1d,
               'maxpool2d':nn.MaxPool2d,
               'maxpool3d':nn.MaxPool3d,
               'sigmoid':nn.Sigmoid(),
               'tanh':nn.Tanh(),
               'relu':nn.ReLU(),
               'lrelu':nn.LeakyReLU(),
               'bn1d':nn.BatchNorm1d,
               'bn2d':nn.BatchNorm2d,
               'bn3d':nn.BatchNorm3d,
               'dropout':nn.Dropout}

def LayerGroup(mylayers):
    mylayer_list = mylayers.split(';')

    module_list = []
    for mylayer in mylayer_list:
        if '-' in mylayer:
            tmp = mylayer.split('-')

            name = tmp[0]

            if name == 'linear':
                dims = [int(v) for v in tmp[1].split(',')]
                sub_model = TorchModule[name](*dims)
            elif 'conv' in name:
                dims = [int(v) for v in tmp[1].split(',')]
                sub_model = TorchModule[name](*dims)
            elif 'pool' in name:
                dims = [int(v) for v in tmp[1].split(',')]
                sub_model = TorchModule[name](*dims)
            elif 'bn' in name:
                dims = [int(v) for v in tmp[1].split(',')]
                sub_model = TorchModule[name](*dims)
            elif name == 'dropout':
                dims = [float(v) for v in tmp[1].split(',')]
                sub_model = TorchModule[name](*dims)

            module_list.append(sub_model)

        else:
            module_list.append(TorchModule[mylayer])

    module_list = nn.ModuleList(module_list)
    return nn.Sequential(*module_list)

# Examples with this module function.

In [ ]:
# Simple neural network; (B,200) - (B,10) - relu
# linear-[in][out]

layer_text = 'linear-200,10;relu'
model = LayerGroup(layer_text).cuda()
print(model)

x = torch.randn(10,200).cuda()
y = model(x)
print(y.shape)

In [ ]:
# Neural network; (B,100) - (B,50) - (B,10) - (B,1) (relu as activation function)
layer_text = 'your code here'
model = LayerGroup(layer_text).cuda()
print(model)

x = torch.randn(10,100).cuda()
y = model(x)
print(y.shape)

In [ ]:
# Simple convolutional neural network; (B,3,32,32) - (B,32,16,16)
# conv2d-[in][out][kernel][stride][padding]

layer_text = 'conv2d-3,32,4,2,1;lrelu'
model = LayerGroup(layer_text).cuda()
print(model)

x = torch.randn(10,3,32,32).cuda()
y = model(x)
print(y.shape)

In [ ]:
# Convolution operations; (B,3,32,32) - (B,32,16,16) - (B,64,8,8) - (B,128,4,4) (lrelu as activation function)
layer_text = 'your code here'
model = LayerGroup(layer_text).cuda()
print(model)

x = torch.randn(10,3,32,32).cuda()
y = model(x)
print(y.shape)

In [ ]:
# Convolution operations; (B,3,32,32) - BatchNorm - (B,32,16,16) - BatchNorm - (B,64,8,8) - BatchNorm - (B,128,4,4) (add lrelu after BatchNorm)
# You don't need to write all model information in single line
# Define each layer module and then combine it!
# bn2d-[dim] ; batch normalization 2D [channel dimension]

conv_text1 = 'conv2d-3,32,4,2,1;bn2d-32;lrelu'
conv_text2 = 'conv2d-32,64,4,2,1;bn2d-64;lrelu'
conv_text3 = 'conv2d-64,128,4,2,1;bn2d-128;lrelu'
model = LayerGroup(';'.join([conv_text1,conv_text2,conv_text3])).cuda()
print(model)

x = torch.randn(10,3,32,32).cuda()
y = model(x)
print(y.shape)

In [ ]:
# Convolution operations; (B,3,32,32) - (B,32,16,16) - (B,64,8,8) - (B,128,4,4) - (B,256,1,1)
# Use relu activation function between convolution operations
# Apply batch normalization 2D after relu activation function
# Use sigmoid function after the last convolution layer

conv_text1 = 'your code here'
conv_text2 = 'your code here'
conv_text3 = 'your code here'
conv_text4 = 'your code here'

model = LayerGroup(';'.join([conv_text1,conv_text2,conv_text3,conv_text4])).cuda()
print(model)

x = torch.randn(10,3,32,32).cuda()
y = model(x)
print(y.shape)

In [ ]:
# Combine multiple operations (Convolution -> Fully connected neural network)
# Convolution; (B,3,32,32) - (B,32,16,16) - (B,64,8,8) - (B,128,4,4) (add lrelu after BatchNorm)
# Each convolution block; convolution 2D -> batch normalization 2D -> lrelu activation
# FCNN; (B,128*4*4) - (B,512) - (B,128) - (B,5) (relu between linear layer)

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()

        conv_text1 = 'your code here'
        conv_text2 = 'your code here'
        conv_text3 = 'your code here'
        self.conv_block =  LayerGroup(';'.join([conv_text1,conv_text2,conv_text3]))

        fc_text = 'your code here'
        self.fcnn = LayerGroup(fc_text)

    def forward(self,x):
        x = self.conv_block(x)
        x = x.view(-1,128*4*4)
        x = self.fcnn(x)
        return x

my_model = MyModel().cuda()
print(my_model)

x = torch.randn(10,3,32,32).cuda()
y = my_model(x)
print(y.shape)

# Example codes on AE, VAE, GAN

# Autoencoder

In [ ]:
# Example for autoencoder with linear layers

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()

        enc_info = 'linear-784,512;relu;linear-512,20'
        dec_info = 'linear-20,512;relu;linear-512,784;sigmoid'

        self.encoder = LayerGroup(enc_info)
        self.decoder = LayerGroup(dec_info)

    def forward(self,x):
        z = self.encoder(x)
        out = self.decoder(z)
        return z,out

AE = Autoencoder().cuda()
print(AE)

x = torch.randn(10,784).cuda()
z,x_pred = AE(x)
print(x.shape)
print(z.shape)
print(x_pred.shape)

In [ ]:
# Example for autoencoder with linear layers

class Autoencoder(nn.Module):
    def __init__(self,enc_info,dec_info):
        super(Autoencoder,self).__init__()

        self.encoder = LayerGroup(enc_info)
        self.decoder = LayerGroup(dec_info)

    def forward(self,x):
        z = self.encoder(x)
        out = self.decoder(z)
        return z,out

enc_info = 'linear-784,512;relu;linear-512,20'
dec_info = 'linear-20,512;relu;linear-512,784;sigmoid'

AE = Autoencoder(enc_info,dec_info).cuda()
print(AE)

x = torch.randn(10,784).cuda()
z,x_pred = AE(x)
print(x.shape)
print(z.shape)
print(x_pred.shape)

In [ ]:
# Example for autoencoder with linear layers

class Autoencoder(nn.Module):
    def __init__(self,dim_x,dim_h,dim_z):
        super(Autoencoder,self).__init__()

        self.enc_info = f'linear-{dim_x},{dim_h};relu;linear-{dim_h},{dim_z}'
        self.dec_info = f'linear-{dim_z},{dim_h};relu;linear-{dim_h},{dim_x};sigmoid'

        self.encoder = LayerGroup(self.enc_info)
        self.decoder = LayerGroup(self.dec_info)

    def forward(self,x):
        z = self.encoder(x)
        out = self.decoder(z)
        return z,out

dim_x = 28*28
dim_h = 512
dim_z = 64

AE = Autoencoder(dim_x,dim_h,dim_z).cuda()
print(AE)

x = torch.randn(10,784).cuda()
z,x_pred = AE(x)
print(x.shape)
print(z.shape)
print(x_pred.shape)

In [ ]:
# Write code for autoencoder training with MNIST dataset

# Model construction
# Encoder; (B,28*28) - (B,dim_h) - (B,dim_z)
# Decoder; (B,dim_z) - (B,dim_h) - (B,28*28)
# Use relu activation function between linear layer for both encoder & decoder

class Autoencoder(nn.Module):
    def __init__(self,dim_x,dim_h,dim_z):
        super(Autoencoder,self).__init__()

        self.enc_info = 'your code here'
        self.dec_info = 'your code here'

        self.encoder = LayerGroup(self.enc_info)
        self.decoder = LayerGroup(self.dec_info)

    def forward(self,x):
        z = self.encoder(x)
        out = self.decoder(z)
        return z,out

dim_x = 28*28
dim_h = 512
dim_z = 64

AE = Autoencoder(dim_x,dim_h,dim_z).cuda()

# dataset & dataloader construction
training_data = datasets.MNIST(root="./",train=True,download=True,transform=ToTensor())
test_data = datasets.MNIST(root="./",train=False,download=True,transform=ToTensor())

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle=True)

#note; shape of img = [1,28,28], shape of target = [1]

# training code
optimizer = torch.optim.Adam(AE.parameters(), lr=0.001)

num_epoch = 10
Loss = []

for epoch in range(num_epoch):
    print('============= Epoch: '+str(epoch+1)+' =============')
    AE.train()

    L1 = 0
    N1 = 0
    for x,y in tqdm(train_dataloader):
        x = x.cuda().view(-1,28*28)

        z,x_pred = AE(x)

        loss = 'your code here'

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        L1 += len(x)*loss.detach().cpu().numpy()
        N1 += len(x)

    AE.eval()
    Zs = []
    Xs = []
    Ys = []

    L2 = 0
    N2 = 0
    for x,y in tqdm(test_dataloader):
        x = x.cuda().view(-1,28*28)

        z,x_pred = AE(x)

        loss = 'your code here'

        L2 += len(x)*loss.detach().cpu().numpy()
        N2 += len(x)

        Zs.append(z.detach().cpu().numpy())
        Xs.append(x_pred.detach().cpu().numpy())
        Ys.append(y.detach().cpu().numpy())

    Loss.append([L1/N1,L2/N2])
    print(Loss[-1])

Loss = np.array(Loss)
Zs = np.vstack(Zs)
Xs = np.vstack(Xs)
Ys = np.hstack(Ys)

plt.figure(dpi=100)
plt.plot(Loss[:,0],'o--',label='train')
plt.plot(Loss[:,1],'o--',label='test')
plt.legend()

plt.figure(dpi=100)
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(Xs[i].reshape(28,28),cmap='gray')

# Convolution Autoencoder

In [ ]:
# Write code for convolutional autoencoder training with MNIST dataset

# Model construction
class ConvAE(nn.Module):
    def __init__(self,dim_c,dim_z):
        super(ConvAE,self).__init__()
        self.dim_z = dim_z

        self.enc_info = ['your code here', # [dim_c,14,14]
                         'your code here', # [2*dim_c,7,7]
                         'your code here'] # [dim_z,1,1]

        self.dec_info = ['your code here', # [2*dim_c,7,7]
                         'your code here', # [dim_c,14,14]
                         'your code here'] # [1,28,28] (add sigmoid function after last conv layer)

        self.encoder = LayerGroup(';'.join(self.enc_info))
        self.decoder = LayerGroup(';'.join(self.dec_info))

    def forward(self,x):
        z = self.encoder(x)
        out = self.decoder(z)
        return z,out

dim_c = 32
dim_z = 32

AE = ConvAE(dim_c,dim_z).cuda()

print(AE)

# dataset & dataloader construction
training_data = datasets.MNIST(root="./",train=True,download=True,transform=ToTensor())
test_data = datasets.MNIST(root="./",train=False,download=True,transform=ToTensor())

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle=True)

#note; shape of img = [1,28,28], shape of target = [1]

# training code
optimizer = torch.optim.Adam(AE.parameters(), lr=0.0003)

num_epoch = 10
Loss = []

for epoch in range(num_epoch):
    print('============= Epoch: '+str(epoch+1)+' =============')
    AE.train()

    L1 = 0
    N1 = 0
    for x,y in tqdm(train_dataloader):
        x = x.cuda()

        z,x_pred = AE(x)

        loss = torch.mean((x_pred-x)**2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        L1 += len(x)*loss.detach().cpu().numpy()
        N1 += len(x)

    AE.eval()
    Zs = []
    Xs = []
    Ys = []

    L2 = 0
    N2 = 0
    for x,y in tqdm(test_dataloader):
        x = x.cuda()

        z,x_pred = AE(x)

        loss = torch.mean((x_pred-x)**2)

        L2 += len(x)*loss.detach().cpu().numpy()
        N2 += len(x)

        Zs.append(z.detach().cpu().numpy())
        Xs.append(x_pred.detach().cpu().numpy())
        Ys.append(y.detach().cpu().numpy())

    Loss.append([L1/N1,L2/N2])
    print(Loss[-1])

Loss = np.array(Loss)
Zs = np.vstack(Zs)
Xs = np.vstack(Xs)
Ys = np.hstack(Ys)

plt.figure(dpi=100)
plt.plot(Loss[:,0],'o--',label='train')
plt.plot(Loss[:,1],'o--',label='test')
plt.legend()

plt.figure(dpi=100)
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(Xs[i].reshape(28,28),cmap='gray')

# Variational Autoencoder with CNN

In [ ]:
# Write code for convolutional autoencoder training with MNIST dataset

# Model construction
class ConvVAE(nn.Module):
    def __init__(self,dim_c,dim_z):
        super(ConvVAE,self).__init__()
        self.dim_z = dim_z

        self.enc_info = ['your code here', # [dim_c,14,14]
                         'your code here', # [2*dim_c,7,7]
                         'your code here' # [2*dim_z,1,1]

        self.dec_info = ['your code here', # [2*dim_c,7,7]
                         'your code here', # [dim_c,14,14]
                         'your code here'] # [1,28,28] (add sigmoid function after last conv layer)

        self.encoder = LayerGroup(';'.join(self.enc_info))
        self.decoder = LayerGroup(';'.join(self.dec_info))

    def forward(self,x):

        nz = self.dim_z
        h = self.encoder(x)

        mu = h[:,:nz]
        logvar = h[:,nz:]

        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps*std

        out = self.decoder(z)
        return z,out,mu,logvar

dim_c = 16
dim_z = 32

AE = ConvVAE(dim_c,dim_z).cuda()

print(AE)

# dataset & dataloader construction
training_data = datasets.MNIST(root="./",train=True,download=True,transform=ToTensor())
test_data = datasets.MNIST(root="./",train=False,download=True,transform=ToTensor())

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle=True)

#note; shape of img = [1,28,28], shape of target = [1]

# training code
optimizer = torch.optim.Adam(AE.parameters(), lr=0.0003)

num_epoch = 10
Loss = []

for epoch in range(num_epoch):
    print('============= Epoch: '+str(epoch+1)+' =============')
    AE.train()

    L1 = 0
    N1 = 0
    for x,y in tqdm(train_dataloader):
        x = x.cuda()

        z,x_pred,mu,logvar = AE(x)

        rec = 'your code here'
        kld = 'your code here'

        loss = rec + 0.015*kld

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        L1 += len(x)*rec.detach().cpu().numpy()
        N1 += len(x)

    AE.eval()
    Zs = []
    Xs = []
    Ys = []

    L2 = 0
    N2 = 0
    for x,y in tqdm(test_dataloader):
        x = x.cuda()

        z,x_pred,mu,logvar = AE(x)

        rec = 'your code here'
        kld = 'your code here'

        L2 += len(x)*rec.detach().cpu().numpy()
        N2 += len(x)

        Zs.append(z.detach().cpu().numpy())
        Xs.append(x_pred.detach().cpu().numpy())
        Ys.append(y.detach().cpu().numpy())

    Loss.append([L1/N1,L2/N2])
    print(Loss[-1])

Loss = np.array(Loss)
Zs = np.vstack(Zs).reshape(-1,dim_z)
Xs = np.vstack(Xs)
Ys = np.hstack(Ys)

plt.figure(dpi=100)
plt.plot(Loss[:,0],'o--',label='train')
plt.plot(Loss[:,1],'o--',label='test')
plt.legend()

plt.figure(dpi=100)
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(Xs[i].reshape(28,28),cmap='gray')

plt.figure(dpi=100)
for i in range(dim_z):
    sns.kdeplot(Zs[:,i])

plt.figure(dpi=100)

z = torch.randn(16,dim_z,1,1).cuda()
x_new = AE.decoder(z).cpu().detach().numpy()

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(x_new[i].reshape(28,28),cmap='gray')

# GAN (Convolutions)

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, conv_dim):
        super(Discriminator, self).__init__()

        self.conv_info = [f'conv2d-1,{conv_dim},4,2,1;lrelu',
                          f'conv2d-{conv_dim},{conv_dim*2},4,2,1;bn2d-{conv_dim*2};lrelu',
                          f'conv2d-{conv_dim*2},{conv_dim*4},4,2,1;bn2d-{conv_dim*4};lrelu',
                          f'conv2d-{conv_dim*4},1,4,2,1'] # What is the expected output dimension at each step?

        self.conv_layer = LayerGroup(';'.join(self.conv_info))

    def forward(self, x):
        B = len(x)

        out = self.conv_layer(x)
        return out.view(-1,1)

class Generator(nn.Module):

    def __init__(self, z_dim, conv_dim):
        super(Generator, self).__init__()

        self.conv_dim = conv_dim

        self.convtrans_info = [f'convtrans2d-{z_dim},{conv_dim*8},4,1,0;bn2d-{conv_dim*8};relu',
                               f'convtrans2d-{conv_dim*8},{conv_dim*4},4,2,1;bn2d-{conv_dim*4};relu',
                               f'convtrans2d-{conv_dim*4},{conv_dim*2},4,2,1;bn2d-{conv_dim*2};relu',
                               f'convtrans2d-{conv_dim*2},{conv_dim},4,2,1;bn2d-{conv_dim};relu',
                               f'convtrans2d-{conv_dim},1,1,1,2;tanh'] # What is the expected output dimension at each step?

        self.convtrans_layer = LayerGroup(';'.join(self.convtrans_info))

    def forward(self, x):
        B = len(x)
        x = x.view(B,-1,1,1)
        out = self.convtrans_layer(x)
        return out

In [ ]:
# dataset & dataloader construction
training_data = datasets.MNIST(root="./",train=True,download=True,transform=ToTensor())
test_data = datasets.MNIST(root="./",train=False,download=True,transform=ToTensor())

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle=True)

z_dim = 100
conv_dim = 64

D = Discriminator(conv_dim).cuda()
G = Generator(z_dim,conv_dim).cuda()

print(D)
print(G)

beta1 = 0.5
beta2 = 0.999

optim_D = torch.optim.Adam(D.parameters(),lr=0.0002,betas=[beta1,beta2])
optim_G = torch.optim.Adam(G.parameters(),lr=0.0002,betas=[beta1,beta2])

criterion = nn.BCEWithLogitsLoss()

num_epoch = 10
smooth = True

Loss = []
D.train()
G.train()
for epoch in range(num_epoch):
    print('============= Epoch: '+str(epoch+1)+' =============')

    for batch_i,(real_img,_) in enumerate(train_dataloader):

        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================

        batch_size = real_img.shape[0]

        real_img = 2*real_img-1
        real_img = real_img.cuda()

        optim_D.zero_grad()

        D_real = D(real_img)

        if smooth:
            labels = 0.9*torch.ones(batch_size).cuda()
        else:
            labels = torch.ones(batch_size).cuda()

        loss_real = criterion(D_real.view(-1),labels)

        with torch.no_grad():
            z = torch.randn(batch_size,z_dim).cuda()
            fake_img = G(z)

        D_fake = D(fake_img)
        labels = torch.zeros(batch_size).cuda()
        loss_fake = criterion(D_fake.view(-1),labels)

        loss_D = loss_real + loss_fake
        loss_D.backward()
        optim_D.step()

        # =========================================
        #            TRAIN THE GENERATOR
        # =========================================

        optim_G.zero_grad()

        z = torch.randn(batch_size,z_dim).cuda()
        fake_img = G(z)

        D_fake = D(fake_img)

        if smooth:
            labels = 0.9*torch.ones(batch_size).cuda()
        else:
            labels = torch.ones(batch_size).cuda()

        loss_G = criterion(D_fake.view(-1),labels)
        loss_G.backward()
        optim_G.step()

        if batch_i % 400 == 0:
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epoch, loss_D.item(), loss_G.item()))

In [ ]:
G.eval()

sample_size=16
z = torch.randn(sample_size,z_dim).cuda()
samples = G(z).cpu().detach().numpy()

for i in range(16):
    plt.subplot(4,4,i+1)

    img = np.transpose(samples[i],(1,2,0))
    img = (img+1)/2

    plt.imshow(img,cmap='gray')